<a href="https://colab.research.google.com/github/samarasrs/CTC34/blob/main/CTC34_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Projeto CTC-34**</center>
## <center>Evolução Gramatical</center>

Autores: 

*   Carlos Renato de Andrande Figueiredo;
*   Matheus da Silva Martins; e
*   Samara Ribeiro Silva.

---

A CONSTANTE PODE ASSUMIR UM ÚNICO VALOR? PODERIA TER FEITO UMA GRAMÁTICA DE DÍGITOS. TB NÃO ENTENDI PQ A OPERAÇÃO COM CONSTANTE TEM QUE SER PARENTIZADA.

## Gramática utilizada

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Especificar a gramática livre de contexto utilizada, com os componentes G=(V,$\Sigma$,P,S).

V = {< exp >, < op >, < pre_op >, < var >, < const >}

$\Sigma$ = { Log, Sqrt, +, -, *, /, x1, x2, x3, x4, x5, x6, x7, x8, const,() }

S = { < exp > }

P = { < exp > ::= < exp > < op > < exp > <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| ( < exp > < op > < const > )<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|  < pre_op > ( < exp > ) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|  < var >  <br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; < op >::= + <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| - <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| *<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| /<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; < pre_op >::= Log <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Sqrt<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; < var >::= x1 | x2 | x3 | x4 | x5 | x6 | x7 | x8 <br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; < const >::= c

---

## Códigos Desenvolvidos

Apresentar os códigos desenvolvidos no projeto

Importando Bibliotecas

In [ ]:
# Importando Bibliotecas
import pandas as pd
import math as m
import random as r
from statistics import mean
import numpy as np


Lendo dados

In [ ]:
# Lendo dados do arquivo de treinamento
data_train = pd.read_csv("train.csv", sep=',', decimal='.')

# criando variáveis
ID = data_train["id"].tolist()

desejado_MHV = data_train["median_house_value"].tolist()

x1_Train = data_train["median_income"].tolist()
x2_Train = data_train["housing_median_age"].tolist()
x3_Train = data_train["total_rooms"].tolist()
x4_Train = data_train["total_bedrooms"].tolist()
x5_Train = data_train["population"].tolist()
x6_Train = data_train["households"].tolist()
x7_Train = data_train["latitude"].tolist()
x8_Train = data_train["longitude"].tolist()

Criando Funções antes de rodar o algoritmo:

*   Gerando a população
*   Realizando a derivação



In [ ]:
# Gerando População a partir das definições de quantidade
# de genes e cromossomos
# O valor de cada gene varia de 0 a 256
def generatePopulation(size_cromo, size_pop):
  pop = []
  for i in range(0, size_pop ):
    cromo = []
    for i in range(0, size_cromo):
      cromo.append(r.randint(0,256))
    pop.append(cromo)
  return pop

# Escolha da Regra
def chooseRule(expre, index, cromo):
  # const é um valor aleatorio entre 0 e 256 dividido por 10
  if expre == "const":
    if index>1:
      aux = r.randint(0,256)/10
    else:
      expre = "var"
      aux = rule[expre][cromo[index%len(cromo)]%len(rule[expre])]
  else: 
    aux = rule[expre][cromo[index%len(cromo)]%len(rule[expre])]
  return aux

#Gerando uma expressao
def makeDerivations(cromo, quant_iteracao):
  index = 0
  resp = ""
  expre = ["exp"]
  cont_parent = 0
  while len(expre)>0 and index <quant_iteracao:
    if (expre[0] == "exp" or expre[0] == "op" or expre[0] == "pre_op" or expre[0] == "var"or expre[0] == "const"):
      aux = chooseRule(expre[0], index, cromo)
      index= index+1
      if expre[0] == "exp":
        aux2 = list(aux)
        expre.pop(0)
        for i in range(len(aux2)-1,-1,-1):
          expre.insert(0, aux2[i])
      else:
        resp=resp+str(aux)
        expre.pop(0)
    else:
      resp=resp+expre[0]
      if expre[0]=="(":
        cont_parent = cont_parent + 1
      elif expre[0]==")":
        cont_parent = cont_parent - 1
      expre.pop(0)
  for i in range(0, cont_parent):
      resp = resp +")"
  return resp

#Gerando lista de expressões
def makList(pop, size_pop, quant_iteracao):
  express_list = []
  for i in range(0,size_pop):
    aux = makeDerivations(pop[i], quant_iteracao)
    express_list.append(aux)
  return express_list

Calculando o erro médio quadrático

In [ ]:
# Calculando o erro quadratico medio e printando na tela
def saveExpressMSE(list_express, k):
  mse = [] #MSE = MEAN SQUARED ERROR
  print("*************************************")
  print(k)
  for j in range(0, len(list_express)):
    erro = False
    #mostra na tela qual a iteracao que se encontra
    if j%mostrar==0:
      print(j, end=" ")
    se_ind = []
    aux = list_express[j]
    for i in range(0, len(data_train)):
      x1=float(x1_Train[i])
      x2=float(x2_Train[i])
      x3=float(x3_Train[i])
      x4=float(x4_Train[i])
      x5=float(x5_Train[i])
      x6=float(x6_Train[i])
      x7=float(x7_Train[i])
      x8=float(x8_Train[i])
      desejado=float(desejado_MHV[i])

      try:
        calculated = float(eval(str(aux)))
      except:
        erro =True
        calculated = 0

      try:
        se =(calculated- desejado)**2
      except:
        se = 999999

      se_ind.append(se) 

    if erro:
      mse.append(999999)
    else:    
      mse.append(mean(se_ind))
  return mse
  

Criando Funções:

*   Torneio
*   Cruzamento
*   Mutação

In [ ]:
# Fazendo o torneio entre dois cromossomos
def fight(mse):
  winners = []
  for i in range(0, len(mse)):
    player1=r.randint(0,len(mse)-1)
    player2=r.randint(0,len(mse)-1)

    if mse[player1]<=mse[player2]:
      winners.append(player1)
    else:
      winners.append(player2)
  return winners
        
# Cruzamento de dois cromossomos
def crossing(winners, pop, prob_cruz):
  new_pop = []
  for i in range(0, len(winners),2):
    aux = r.random()
    if aux < prob_cruz:
      ponto_cruz = r.randint(1,len(pop[i])-1)
      child1 = pop[winners[i]][0:ponto_cruz]+pop[winners[i+1]][ponto_cruz:len(pop[i])]
      child2 = pop[winners[i+1]][0:ponto_cruz]+pop[winners[i]][ponto_cruz:len(pop[i])]
      new_pop.append(child1)
      new_pop.append(child2)
    else:
      new_pop.append(pop[i])
      new_pop.append(pop[i+1])
  return new_pop

# Gerando Mutacoes
def mutation(pop, prob_mut):
  for i in range(0, len(pop)):
    for j in range(0, len(pop[i])):
      aux=r.random()
      if aux < prob_mut:
        pop[i][j]= r.randint(0,256)

  return pop

Função Main

In [ ]:
#Rodando
def run(size_pop, size_cromo, quant_iteracao):
  pop = generatePopulation(size_cromo, size_pop)
  bests_mse = []
  bests_express = []
  for i in range(0,quant_geracoes):
    express_list = makList(pop, size_pop, quant_iteracao)
    mse = saveExpressMSE(express_list, i)
    aux = min(mse)
    bests_mse.append(aux)
    bests_express.append(express_list[mse.index(aux)])
    print("\n" ,i, " - mse: ",bests_mse[i], " - express: ", bests_express[i])
    torneio = fight(mse)
    pop = crossing(torneio, pop, prob_cruz)
    pop = mutation(pop, prob_mut)

  bestInd_mse = min(bests_mse)
  bestInd_express = bests_express[bests_mse.index(bestInd_mse)]
  print(bestInd_mse)
  print(bestInd_express)
  return [bestInd_mse, bestInd_express]

Constantes, Gramática e chamada da função Main

In [ ]:
#Regras da Gramatica
# expressoes: <exp> = <exp><op><exp> |
#                     (<exp><op><const>) |
#                     <pre_op>(<exp>)|
#                     <var>
#<op> = +,-,/,*
#<pre_op> = sqrt, log
#<var> =  x1|x2|x3|x4|x5|x6|x7|x8
rule ={"exp" : [["exp", "op", "exp"],
                ["(","exp", "op", "const",")"], 
                ["pre_op","(","exp",")"],
                ["var"]
                
                ],
       
        "op" : ["+", "-", "/", "*"],
       
        "pre_op" : ["m.sqrt","m.log"],
       
        "var" : ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]}

#Constantes
# Quantidade de genes
size_cromo = 25
# Quantidade de cromossomos
size_pop =1500
# Limite de Iterações pra formar uma expressão
quant_iteracao = 20
# Probabilidade de Cruzamento
prob_cruz = 0.8
# Probabilidade de Mutação
prob_mut = 0.1
# Quantidade de Gerações
quant_geracoes = 50
# Quantidade de iterações para mostrar na tela
mostrar = 50
 
# Rodar o código  
resp = run(size_pop, size_cromo, quant_iteracao)
# Printando a resposta encontrada
print(resp)

ESSE 10.8 É O VALOR DA CONSTANTE? COMO AJUSTOU ELE?

### Melhor Expressão Encontrada:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
MSE = 0.1865131837069543

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Expressão:(log(x1)+10.8)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
(x1 corresponde a "median_income")

Lendo dados do arquivo Test

In [ ]:
# Lendo arquivo do teste
data_teste = pd.read_csv("test.csv", sep=',', decimal='.')

ID = data_teste["id"].tolist()

# De acordo com o melhor valor encontrado, precisamos apenas de x1:
x1_Test = data_teste["median_income"].tolist()


Gerando Resposta

In [ ]:
# Gerando resposta
resposta = []

for i in range(len(x1_Test)):
  x1=float(x1_Test[i])
  media = (m.log(x1)+10.8)
  resposta.append(media)


Gravando a resposta no arquivo resultado

In [ ]:
# Convertendo resposta para dataframe e depois export para csv
dataframe_valores = {
    'id': ID,
    "median_house_value":resposta
}

df = pd.DataFrame(dataframe_valores, columns = ['id',"median_house_value"])

df.to_csv(r'resultado.csv',index= False)

         id  median_house_value
0         4           12.530434
1         5           12.147086
2         8           11.937833
3         9           11.532560
4        10           12.105952
...     ...                 ...
6186  20621           12.317871
6187  20626           12.217066
6188  20627           11.573205
6189  20629           11.753664
6190  20637           11.738756

[6191 rows x 2 columns]


---

OK, ÓTIMO

## Experimentos

O algoritmo foi executado diversas vezes variando o tamanho da população e a quantidade de gerações. Para a resposta final foi utilizada o tamanho da população 1500 e 50 gerações. Utilizamos também um parâmetro de quantidade de iteração da derivação de 20 para evitar que as expressões ficassem muito longas ou entrasse em looping infinito.

---


## Resultados

Apresentar os resultados alcançados:


*   Qual foi a função de regressão final?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Expressão:&nbsp;&nbsp; (log(x1)+10.8)
*   Que erro médio quadrático é alcançado no conjunto de treinamento?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;MSE = 0.1865131837069543
*   Que erro médio quadrático é alcançado no conjunto de teste após submissão no Kaggle?

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Score = 0.18546

---


PARABÉNS PELO TRABALHO, GOSTEI. MINHA SUGESTÃO QUANTO À CONSTANTE É FAZER A GRAMÁTICA PARA GERÁ-LA. ASSIM O PRÓPRIO ALGORITMO EVOLUTIVO ENCONTRA QUAL O MELHOR VALOR DE CONSTANTE. DÁ PARA ESCREVER A GRAMÁTICA DE OUTRAS FORMAS TAMBÉM, NA DISCIPLINA DE COMPILADORES VOCÊS PROVAVELMENTE DEVEM ESTUDAR ASSUNTOS QUE AJUDAM NESSE SENTIDO.

## Conclusão

O resultado obtido foi satisfatório, portanto o algoritmo foi adequado para o problema. 

Para melhorar o código o valor da constante $c$ poderia ter mais casas decimais podendo assim melhorar a precissão do resultado. 


No nosso código, todas as expressões que não podiam ser calculadas utilizando a função $eval$ eram desconsideradas. Com isso, outra maneira de aperfeiçoar o código seria fazer alguma implementação que diminuísse o número de expressões inválidas de maneira mais eficiente, isso poderia ocasionar uma possível diminuição no tamanho da população necessária para encontrar uma resposta aceitável.

---